# Instalação de bibliotecas

In [258]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [259]:
 pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [260]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [261]:
pip install enelvo

Note: you may need to restart the kernel to use updated packages.


In [262]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [263]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [264]:
import pandas as pd
from textblob import TextBlob
import emoji
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
nltk.download('wordnet')
import re
from enelvo.normaliser import Normaliser
import matplotlib.pyplot as plt


import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Abrindo o dataframe inicial

In [265]:
df = pd.read_excel('../base/base.xlsx', index_col=0)

In [266]:
df

,"""dataPublicada""","""autor""","""texto""","""sentimento""","""tipoInteracao""","""anomalia""","""probabilidadeAnomalia""","""linkPost""","""processado""","""contemHyperlink"""
id,,,,,,,,,,
1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0
...,...,...,...,...,...,...,...,...,...,...
3044013,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0,0
3044212,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0,0
3044213,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0


# Limpeza dos dados

In [267]:
# Esta função remove os dados que não serão utilizados para a análise
def clean_data(df):
    # Remove as aspas dos nomes das colunas
    df = df.rename(columns=lambda x: x.replace('\"', ''))
    # Remove comentários feito pelo btg
    dfWithoutAutor = df[df["autor"] != "btgpactual"]
    # Remove comentários nulos
    dfWithoutNull = dfWithoutAutor[dfWithoutAutor["autor"].notnull()]
    # Deixando apenas as interações do tipo comentário e resposta
    dfFinal = dfWithoutNull.loc[(dfWithoutNull['tipoInteracao'] == 'comentário') | (dfWithoutNull['tipoInteracao'] == 'resposta')]
    return dfFinal

In [268]:
cleaning_output = clean_data(df)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
id,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0
...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0


# Pré processamento

## Tratamento de emojis

In [269]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [270]:
pip install pandas numpy emoji

Note: you may need to restart the kernel to use updated packages.


In [271]:
emoji_dict = {
    '😀': 'feliz ',
    '😂': 'risos ',
    '😔': 'triste ',
    '👏': 'palmas ',
    '🥰': 'amável ',
    '💙': 'coração azul ',
    '🙏🏼': 'orando ',
    '✨': 'brilhando ',
    '🤮': 'nojo ',
    '🚀': 'foguete ',
    '👿': 'diabo ',
    '🤢': 'nojo ',
    '🔥': 'fogo ',
    '😡': 'fúria ',
    '😠': 'raiva ',
    '🤣': 'rindo ',
    '😃': 'feliz ',
    '😎': 'curtindo ',
    '😊': 'feliz ',
    '🤩': 'maravilhado ',
    '😋': 'delicioso ',
    '😆': 'risada ',
    '😌': 'calmo ',
    '🤔': 'pensativo ',
    '😷': 'máscara ',
    '🤣': 'muitoRiso ',
    '🥺': 'carinhoso ',
    '👍': 'positivo ',
    '🤯': 'menteExplodida ',
    '😅': 'alívio ',
    '🥰': 'carinhaComCoração ',
    '😓': 'suor ',
    '😑': 'tédio',
    '🤫': 'silêncio',
    '🤝': 'apertoDeMãos',
    '😊': 'sorriso',
    '😍': 'apaixonado',
    '😭': 'choro ',
    '🤗': 'abraço ',
    '🎉': 'festa ',
    '😎': 'descolado ',
    '😱': 'surpresa ',
    '😴': 'sono ',
    '🙌': 'celebração ',
    '🤔': 'pensativo ',
    '😘': 'beijo ',
    '🥳': 'festeiro ',
    '🙄': 'revirarOsOlhos ',
    '😌': 'alívio ',
    '🤫': 'segredo ',
    '😇': 'inocente ',
    '😂': 'muitoEngraçado ',
    '🤔': 'pensando ',
    '😴': 'sono ',
    '🤪': 'loucura ',
    '😢': 'decepcionado_mas_aliviado ',
}

In [272]:
def emoji_to_word(text):
    for emoji_code, emoji_word in emoji_dict.items():
        text = text.replace(emoji_code, emoji_word)
    return text

In [273]:
cleaning_output['texto_e_palavra'] = cleaning_output['texto'].apply(emoji_to_word)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra
id,,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas
...,...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus


In [274]:
# Testando a função 
processing_test = cleaning_output['texto']
emoji_test = processing_test.apply(emoji_to_word)

In [275]:
cleaning_output['texto_e_palavra'].head(20)

id
4                                  Minuto touro de ouro
5                                      @ricktolledo Sim
6     Queria saber se a Btg banking é a própria btg ...
11                                       @thaotinhasbfc
18                         palmas palmas palmas palmas 
19    Vamos ajudar o Brasil na principal categoria d...
20    Com tanta conta digital dando baile nestes ban...
22    @hericles.sisnando o Instagram só deixa public...
23     Cadê a de dividendos? decepcionado_mas_aliviado 
25    Estou aqui apenas para mostrar meu apreço à mu...
26                                coração azul foguete 
31    Eita que não existe ninguém pra responder nem ...
32    Eu mandei pergunta no privado e vcs não me res...
33            Como alocar  valor nessa nova atualização
34    @elson.linss eu recomendo o maior banco de inv...
35                                           Disciplina
37      @gwm.investments e @btgpactual muito alinhados!
43                     Qual horário especial 

## Tokenização e remoção de stop words

In [291]:
def processarTexto(text):
    # Converter palavras da frase para minúsculas
    text = text.lower() 
    # Tokenizar o texto em palavras
    tokens = word_tokenize(text) 
    # Remover stopwords das palavras tokenizadas
    inicial_stop_words = stopwords.words('portuguese') 
    custom_stop_words = ['banco', 'btg', 'brg', 'pactual', 'btgpactual','pq', 'q', 'pra', 'vcs', 'vc', 'i', 'p', 'kkk', 'y', 'of', 'n']
    stop_words = inicial_stop_words + custom_stop_words
    # Retorna a lista de comentários sem as palavras setadas para serem removidas
    tokens = [token for token in tokens if token not in stop_words and not token.startswith('@') and token.isalpha()] 
    # Retorna o resultado da frase tokenizada, sem stopWords
    return tokens

In [292]:
#Aplicar a função em uma única variável para aplicar nos modelos
tokens_processados = cleaning_output['texto'].apply(processarTexto)
tokens_processados

id
4                                      [minuto, touro, ouro]
5                                         [ricktolledo, sim]
6              [queria, saber, banking, própria, administra]
11                                           [thaotinhasbfc]
18                                                        []
                                 ...                        
3042413                              [excelente, explicação]
3042713    [porque, tá, diminuído, limite, cartão, sempre...
3043812                      [atendam, telefone, amor, deus]
3044213    [grato, todo, esforço, rendeu, investimento, r...
3045012    [morning, call, aparecendo, spotify, atualizad...
Name: texto, Length: 6356, dtype: object

In [293]:
cleaning_output['tokens_processados'] = cleaning_output['texto_e_palavra'].apply(processarTexto)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,tokens_processados,positivo,negativo,neutro
id,,,,,,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"[minuto, touro, ouro]",0.0,0.0,1.0
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,"[ricktolledo, sim]",0.0,0.0,1.0
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"[queria, saber, banking, própria, administra]",0.0,0.0,1.0
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc,[thaotinhasbfc],0.0,0.0,1.0
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"[palmas, palmas, palmas, palmas]",0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"[excelente, explicação]",NaN,NaN,NaN
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"[porque, tá, diminuído, limite, cartão, sempre...",NaN,NaN,NaN
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"[atendam, telefone, amor, deus]",NaN,NaN,NaN


In [249]:
cleaning_output = cleaning_output.drop(cleaning_output[cleaning_output.tokens_processados == '[]'].index)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,tokens_processados,texto_e_palavra
id,,,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,"[minuto, touro, ouro]",Minuto touro de ouro
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,"[ricktolledo, sim]",@ricktolledo Sim
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,"[queria, saber, banking, própria, administra]",Queria saber se a Btg banking é a própria btg ...
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,[thaotinhasbfc],@thaotinhasbfc
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,[],palmas palmas palmas palmas
...,...,...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,"[excelente, explicação]",Excelente explicação
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,"[porque, tá, diminuído, limite, cartão, sempre...",Porque o brg tá diminuído o limite do cart...
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,"[atendam, telefone, amor, deus]",Atendam o telefone pelo amor de Deus


# Aplicação Bag of Words

Durante esta etapa é feita a implementação do modelo Bag of Words, que vetoriza as palavras e faz a contagem de vezes que cada uma aparece na coluna a ser utilizada. Assim, tendo como resultado um dataframe com as palavras que mais aparecem no texto e sua respectiva frequência.

In [251]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import ast
#A função abaixo obtém o dicionário e os vetores para, posteriormente, ser aplicado nos modelos

def bag_of_words(frases):
    vectorizer = CountVectorizer()
    frases = [' '.join(tokens) for tokens in frases]
    bow_representation = vectorizer.fit_transform(frases)
    dictionary = vectorizer.vocabulary_

    return bow_representation, dictionary

# Aplicar a função de Bag of Words
bow_representation, dictionary = bag_of_words(tokens_processados)
bow_representation, dictionary

(<6356x9530 sparse matrix of type '<class 'numpy.int64'>'
 	with 46210 stored elements in Compressed Sparse Row format>,
 {'minuto': 5931,
  'touro': 8827,
  'ouro': 6504,
  'ricktolledo': 7966,
  'sim': 8302,
  'queria': 7501,
  'saber': 8059,
  'banking': 1043,
  'própria': 7371,
  'administra': 203,
  'thaotinhasbfc': 8709,
  'vamos': 9119,
  'ajudar': 330,
  'brasil': 1273,
  'principal': 7216,
  'categoria': 1563,
  'automobilismo': 934,
  'mundo': 6071,
  'ambev': 466,
  'xpinvestimentos': 9442,
  'havanoficial': 4519,
  'bancodobrasil': 1018,
  'clarobrasi': 1732,
  'nubank': 6300,
  'itau': 5088,
  'santander': 8113,
  'wegelectric': 9407,
  'tanta': 8579,
  'conta': 2161,
  'digital': 2892,
  'dando': 2473,
  'baile': 990,
  'nestes': 6227,
  'bancos': 1023,
  'raios': 7549,
  'continuam': 2181,
  'itaú': 5095,
  'hora': 4579,
  'dar': 2487,
  'adeus': 186,
  'escolha': 3456,
  'bancointer': 1019,
  'feliz': 3921,
  'itaufoi': 5091,
  'ruim': 8037,
  'desde': 2714,
  'sempre':

In [252]:
# Imprimir o dicionário de palavras
print("Dicionário de palavras:")
print(dictionary, "\n")

# Imprimir a representação Bag of Words
print("Representação Bag of Words:")
print(bow_representation.toarray())

Dicionário de palavras:
{'minuto': 5931, 'touro': 8827, 'ouro': 6504, 'ricktolledo': 7966, 'sim': 8302, 'queria': 7501, 'saber': 8059, 'banking': 1043, 'própria': 7371, 'administra': 203, 'thaotinhasbfc': 8709, 'vamos': 9119, 'ajudar': 330, 'brasil': 1273, 'principal': 7216, 'categoria': 1563, 'automobilismo': 934, 'mundo': 6071, 'ambev': 466, 'xpinvestimentos': 9442, 'havanoficial': 4519, 'bancodobrasil': 1018, 'clarobrasi': 1732, 'nubank': 6300, 'itau': 5088, 'santander': 8113, 'wegelectric': 9407, 'tanta': 8579, 'conta': 2161, 'digital': 2892, 'dando': 2473, 'baile': 990, 'nestes': 6227, 'bancos': 1023, 'raios': 7549, 'continuam': 2181, 'itaú': 5095, 'hora': 4579, 'dar': 2487, 'adeus': 186, 'escolha': 3456, 'bancointer': 1019, 'feliz': 3921, 'itaufoi': 5091, 'ruim': 8037, 'desde': 2714, 'sempre': 8207, 'nunca': 6310, 'vai': 9085, 'mudar': 6048, 'instagram': 4910, 'deixa': 2568, 'publicar': 7399, 'imagens': 4663, 'post': 7043, 'então': 3386, 'conseguiram': 2096, 'colacar': 1805, 'dy'

In [285]:
cleaning_output['sentimento'].dtype()

TypeError: 'numpy.dtype[object_]' object is not callable

In [281]:
import pandas as pd

# Converte a matriz esparsa em uma matriz densa
bow_array = bow_representation.toarray()

# Cria um novo DataFrame com os dados do bow_representation e a coluna de sentimentos
bow = pd.DataFrame(bow_array)
bow['sentimento'] = cleaning_output['sentimento']
bow.head(30)
# Salva o DataFrame em um arquivo CSV
# df.to_csv('bow_representation.csv', index=False)


,0,1,2,3,4,5,6,7,8,9,...,9521,9522,9523,9524,9525,9526,9527,9528,9529,sentimento
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POSITIVE
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NEUTRAL
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,POSITIVE
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [286]:
def bow_dataframe(inputUser):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(inputUser)
    # Com essa variavel é possível ver as palavras e contar sua frequência
    wordCount = tokenizer.word_counts    
    dfCountBoW = pd.DataFrame(list(wordCount.items()))
    dfCountBoW.rename(columns={0: "Palavra", 1:"Frequência"}, inplace=True)
    final_df = dfCountBoW.sort_values(by=['Frequência'], ascending=False)
    return final_df

In [294]:
result_output = bow_dataframe(cleaning_output['tokens_processados'])
result_output.head(20)

,Palavra,Frequência
11,palmas,2685
235,muitoengraçado,453
740,limite,418
29,conta,395
387,cartão,364
85,foguete,297
84,azul,260
236,melhor,258
184,agora,258
83,coração,244


#### Caso de teste

In [46]:
# Output da função
bow_test_result.head(5)

,Palavra,Frequência
8,'palmas',2668
98,'banco',475
193,'muitoengraçado',443
613,'limite',363
16,'conta',319


#### Demonstração da função

Nesta etapa será demonstrado o funcionamento da função na base de dados real 

In [55]:
result_output = bow_dataframe(pre_processing['texto_tokenizado'])
result_output.head(10)

,Palavra,Frequência
8,'palmas',2668
98,'banco',475
193,'muitoengraçado',443
613,'limite',363
16,'conta',319
309,'cartão',310
65,'foguete',295
64,'azul',258
63,'coração',239
194,'melhor',228


# Naive bayes

In [224]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
sentimento_numerico = encoder.fit_transform(cleaning_output['sentimento'])

# Dividir os dados em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(bow_representation, sentimento_numerico, test_size=0.2, random_state=42)

# Criar e treinar o modelo Naive Bayes
modelo = MultinomialNB()
modelo.fit(X_treino, y_treino)

# Fazer a predição usando os dados de teste
predicao_numerica = modelo.predict(X_teste)

# Decodificar as classes preditas
predicao = encoder.inverse_transform(predicao_numerica)

# Avaliar a acurácia do modelo
acuracia = (predicao == cleaning_output['sentimento'].iloc[y_teste]).mean()
print('Acurácia:', acuracia)

Acurácia: 0.5707547169811321


In [13]:
result_output.to_csv('..\\output\\bow.csv',encoding='utf-8', index=False, header=True, )